In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
train_df.head()

In [ ]:
#checking for nan values
for i in train_df.columns:
    if train_df[i].isnull().sum()>0:
        print(train_df[i])


In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt
from sklearn import metrics
corr=train_df.corr()
top_features=corr.index
plt.figure(figsize=(10,10))
g=sns.heatmap(train_df[top_features].corr(),annot=True,cmap="RdYlGn")
plt.show(g)

In [ ]:
sns.countplot(train_df["DEATH_EVENT"]).set_title('Death Event Stats')


In [ ]:
sns.countplot(train_df["DEATH_EVENT"],hue=train_df["sex"]).set_title('Death Event distiburtion by gender')


In [ ]:
sns.countplot(train_df["DEATH_EVENT"],hue=train_df["diabetes"]).set_title('Death Event distiburtion by Diabetes')


In [ ]:
train_df_features=train_df.drop("DEATH_EVENT",axis=1)
print(train_df_features.columns)
train_df_output=train_df["DEATH_EVENT"]


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

X_train, X_test, Y_train, Y_test =train_test_split(train_df_features, train_df_output,random_state=1)
cols = list(train_df_features.columns)
model = RandomForestRegressor()
#Initializing RFE model
rfe = RFE(model, 12)             
#Transforming data using RFE
X_rfe = rfe.fit_transform(X_train,Y_train)  
#Fitting the data to model
model.fit(X_rfe,Y_train)              
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)
train_df=train_df[selected_features_rfe]


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
# Create a based model
rf = RandomForestClassifier(500)
X_train, X_test, Y_train, Y_test =train_test_split(train_df, train_df_output,random_state=1,test_size=0.4)
grid_search = RandomizedSearchCV(estimator = rf, param_distributions = param_grid, cv = 25, n_jobs = -1, verbose = 2)
RFG=RandomForestClassifier(n_estimators=500)
RFG.fit(X_train,Y_train)
y_pred = RFG.predict(X_test)
print(metrics.accuracy_score(Y_test,y_pred))
grid_search.fit(X_train,Y_train)
y_pred = grid_search.predict(X_test)
print(metrics.accuracy_score(Y_test,y_pred))




In [ ]:
from sklearn.metrics import plot_roc_curve
ax = plt.gca()
rfc_disp = plot_roc_curve(grid_search, X_test, Y_test, ax=ax, alpha=0.8)
plt.show()

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
adbo=AdaBoostClassifier(base_estimator=RFG)
adbo.fit(X_train,Y_train)
y_pred = adbo.predict(X_test)
print(metrics.accuracy_score(Y_test,y_pred))


In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict

RFG=RandomForestClassifier(n_estimators=200)
cv = KFold(n_splits=10, random_state=42, shuffle=False)
predicted = cross_val_predict(RFG, train_df, train_df_output, cv=50) 
print(metrics.accuracy_score(train_df_output,predicted))
